# Export/Import Fiddler Models

Export complete model definitions from one Fiddler instance and import them to another, with support for:
* Complete model schema with column definitions (types, ranges, categories)
* Model spec (inputs, outputs, targets, metadata, decisions)
* Task configuration and parameters
* Custom features for LLM models (enrichments)
* Baselines (rolling baselines - static baselines require datasets)
* Related assets (segments, custom metrics)
* Cross-instance transfers (different Fiddler instances)
* Model versioning (create new versions of existing models)
* No DataFrame required for import (uses Model constructor pattern)

## Current Support Status

**Fully Supported:**
* Model schema, spec, and task configuration
* Rolling baselines (auto-created)
* Segments (full import with FQL definitions)
* Custom metrics (full import with definitions)

**Partial Support:**
* Static baselines (exported but require manual dataset publishing before creation)

**Not Yet Supported:**
* Alerts (exported for reference but require manual creation due to metric ID mapping)
* Model artifacts (must be manually re-uploaded after import)

## Prerequisites

* Source Fiddler instance URL and API token
* Target Fiddler instance URL and API token (can be same as source)
* Source model must exist (target model will be created)
* `fiddler_utils` package installed (run cell below)

In [ ]:
# Install dependencies
%pip install -q fiddler-client

# Install fiddler_utils from parent directory
# If not already installed, run from repo root: pip install -e .
import sys

sys.path.insert(0, "..")

In [ ]:
import fiddler as fdl
from fiddler_utils import (
    ConnectionManager,
    ModelManager,
)

print(f"Fiddler client version: {fdl.__version__}")
print("fiddler_utils: Successfully imported")

## Configuration

In [ ]:
# Source Fiddler Instance
SOURCE_URL = ""  # e.g., 'https://source.fiddler.ai'
SOURCE_TOKEN = ""
SOURCE_PROJECT_NAME = ""
SOURCE_MODEL_NAME = ""
SOURCE_MODEL_VERSION = ""  # Optional, leave empty for unversioned models

# Target Fiddler Instance (can be same as source)
TARGET_URL = ""  # e.g., 'https://target.fiddler.ai'
TARGET_TOKEN = ""
TARGET_PROJECT_NAME = ""
TARGET_MODEL_NAME = ""  # Leave empty to use the exported model's name
TARGET_MODEL_VERSION = ""  # Optional, leave empty for unversioned models

# Export Options
INCLUDE_BASELINES = True  # Include baseline definitions
INCLUDE_RELATED_ASSETS = True  # Include segments, metrics, alerts

# Import Options
IMPORT_MODE = "create_new"  # 'create_new', 'create_version', or 'update_existing'
VERSION_LABEL = ""  # Required if IMPORT_MODE='create_version'
CREATE_BASELINES = True  # Recreate baselines on target
IMPORT_RELATED_ASSETS = True  # Import segments, metrics, alerts

## Setup Connections and Initialize Manager

In [ ]:
# Setup connection manager for handling multiple instances
conn_mgr = ConnectionManager(log_level="WARNING")
conn_mgr.add("source", url=SOURCE_URL, token=SOURCE_TOKEN)
conn_mgr.add("target", url=TARGET_URL, token=TARGET_TOKEN)

# Initialize ModelManager
model_mgr = ModelManager()

print("✓ Connection manager configured")
print("✓ ModelManager initialized")

## Fetch Source Model

In [ ]:
# Connect to source and get model
with conn_mgr.use("source"):
    source_project = fdl.Project.from_name(SOURCE_PROJECT_NAME)
    source_model_kwargs = {"project_id": source_project.id, "name": SOURCE_MODEL_NAME}
    if SOURCE_MODEL_VERSION:
        source_model_kwargs["version"] = SOURCE_MODEL_VERSION

    source_model = fdl.Model.from_name(**source_model_kwargs)
    print(f"Source model: {source_model.name} (ID: {source_model.id})")
    if source_model.version:
        print(f"  Version: {source_model.version}")

## Model Export and Import Workflow

The following cells demonstrate the complete workflow for exporting and importing models.

**Note:** Import examples are commented out to avoid creating duplicate models during testing.
Uncomment the import cells to actually perform imports.

### Display Model Information

View comprehensive model details including schema, columns, and related assets.

In [ ]:
with conn_mgr.use("source"):
    model_mgr.display_model_info(
        model_id=source_model.id, show_columns=True, show_related_assets=True
    )

### Export Model

Export complete model definition including all related assets.

In [ ]:
print("\n" + "=" * 60)
print("EXPORTING MODEL")
print("=" * 60)

with conn_mgr.use("source"):
    model_export = model_mgr.export_model(
        model_id=source_model.id, include_baselines=True, include_related_assets=True
    )

print(f"\n✓ Exported model: {model_export.name}")
if model_export.version:
    print(f"  Version: {model_export.version}")
print(f"  Task: {model_export.task}")
print(f"  Columns: {len(model_export.columns)}")
print(f"  Baselines: {len(model_export.baselines)}")
print(f"  Custom features: {len(model_export.custom_features)}")

# Show related assets counts
print("\n  Related assets:")
for asset_type, assets in model_export.related_assets.items():
    print(f"    {asset_type}: {len(assets)}")

# Artifact warning
if model_export.has_artifacts:
    print("\n  ⚠️  Model has uploaded artifacts (must be re-uploaded after import)")

# Show sample column info
print("\n  Sample columns (first 3):")
for col in model_export.columns[:3]:
    info = f"    {col.name} ({col.data_type})"
    if col.categories:
        info += f" - {len(col.categories)} categories"
    elif col.min_value is not None:
        info += f" - range: [{col.min_value}, {col.max_value}]"
    print(info)

### Save Model Export to File

Save the exported model definition to JSON file for later use or archival.

In [ ]:
import os

# Create exports directory if it doesn't exist
os.makedirs("exports", exist_ok=True)

# Save to file
export_filename = f"exports/{model_export.name}_export.json"
model_export.to_json(export_filename)

print(f"✓ Saved model export to: {export_filename}")
print(f"  File size: {os.path.getsize(export_filename) / 1024:.1f} KB")

# Can load it back later:
# loaded_export = ModelExportData.from_json(export_filename)

### Import Model to Target Project

Import the exported model to a different project (or instance) as a new model.

**Note:** This example is commented out to avoid creating duplicate models.
Uncomment to actually import.

In [ ]:
print("\n" + "=" * 60)
print("IMPORTING MODEL (create_new mode)")
print("=" * 60)

with conn_mgr.use("target"):
    # Connect to target project and get Project reference
    target_project = fdl.Project.from_name(TARGET_PROJECT_NAME)

    # Import as new model
    # Will fail if model name already exists (safe default)
    imported_model = model_mgr.import_model(
        target_project_id=target_project.id,
        model_data=model_export,
        import_mode="create_new",
        create_baselines=True,
        import_related_assets=True,
    )

    print(f"\n✓ Imported model: {imported_model.name}")
    print(f"  Model ID: {imported_model.id}")
    print(f"  Task: {imported_model.task}")
    print(f"  Columns: {len(imported_model.schema.columns)}")

### Import as New Model Version

Import as a new version of an existing model using Fiddler's model versioning.

**Note:** This example is commented out to avoid creating duplicate versions.

In [ ]:
# Uncomment to create a new version:

# print("\n" + "=" * 60)
# print("IMPORTING MODEL (create_version mode)")
# print("=" * 60)

# with conn_mgr.use("target"):
#     # Import as new version of existing model
#     versioned_model = model_mgr.import_model(
#         target_project_id=target_project.id,
#         model_data=model_export,
#         import_mode="create_version",
#         version_label="v2-automated",  # Required for create_version mode
#         create_baselines=True,
#         import_related_assets=True,
#     )

#     print(f"\n✓ Created new version: {versioned_model.name}")
#     print(f"  Version: {versioned_model.version}")
#     print(f"  Model ID: {versioned_model.id}")

### Model Management Summary

Key advantages of using ModelManager:

In [ ]:
print("\n" + "=" * 60)
print("MODEL MANAGEMENT SUMMARY")
print("=" * 60)

print("\n✅ Advantages of ModelManager:\n")
print("  • Complete model export (schema, spec, task, baselines, related assets)")
print("  • Uses Model constructor (no DataFrame needed for import!)")
print("  • Deterministic and reproducible imports")
print("  • Automatic UUID resolution for related assets")
print("  • Multiple import modes (create_new, create_version, update_existing)")
print("  • Safe defaults (fails if model exists in create_new mode)")
print("  • JSON serialization for archival and version control")
print("  • Display and comparison utilities")

print("\n📊 Export Statistics:\n")
print(f"  Model: {model_export.name}")
print(f"  Columns: {len(model_export.columns)}")
print(f"  Inputs: {len(model_export.spec['inputs'])}")
print(f"  Outputs: {len(model_export.spec['outputs'])}")
print(f"  Targets: {len(model_export.spec['targets'])}")
print(f"  Baselines: {len(model_export.baselines)}")
print(f"  Custom features: {len(model_export.custom_features)}")

total_related = sum(len(assets) for assets in model_export.related_assets.values())
print(f"  Related assets: {total_related} total")
for asset_type, assets in model_export.related_assets.items():
    print(f"    - {asset_type}: {len(assets)}")

print("\n" + "=" * 60)